# Basic Ollama Usage

This notebook demonstrates basic usage of Ollama with Python, including:
- Basic model interaction
- Streaming responses
- Error handling
- Parameter configuration

In [ ]:
import requests
import json
from typing import Generator, Optional

In [ ]:
class OllamaClient:
    def __init__(self, base_url: str = "http://localhost:11434"):
        self.base_url = base_url

    def generate(
        self, prompt: str, model: str = "llama3.1:latest", stream: bool = False, temperature: float = 0.1, top_p: float = 0.9
    ) -> Optional[str]:
        """Generate a response from the model."""
        url = f"{self.base_url}/api/generate"

        payload = {"model": model, "prompt": prompt, "stream": stream, "options": {"temperature": temperature, "top_p": top_p}}

        try:
            response = requests.post(url, json=payload)
            response.raise_for_status()

            if stream:
                return self._handle_stream(response)
            else:
                return response.json()["response"]

        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None

    def _handle_stream(self, response: requests.Response) -> Generator[str, None, None]:
        """Handle streaming responses."""
        for line in response.iter_lines():
            if line:
                try:
                    chunk = json.loads(line)
                    yield chunk["response"]
                except json.JSONDecodeError:
                    continue

## Basic Connectivity

### Check if Ollama is up and running by asking the api version

### Check which models are supported


In [ ]:
import requests


def check_ollama_connectivity(base_url="http://localhost:11434"):
    """
    Checks if the Ollama server is up and running by querying the API version.

    Args:
        base_url (str): The base URL of the Ollama server.

    Returns:
        tuple: A tuple containing a boolean indicating success and a message.
    """
    try:
        response = requests.get(f"{base_url}/api/version")
        response.raise_for_status()  # Raise an exception for bad status codes
        return True, "Ollama server is up and running.", response.json()
    except requests.exceptions.ConnectionError:
        return False, "Failed to connect to Ollama server. Is it running?", None
    except requests.exceptions.RequestException as e:
        return False, f"Error checking Ollama version: {e}", None


def get_available_models(base_url="http://localhost:11434"):
    """
    Retrieves the list of available models from the Ollama server.

    Args:
        base_url (str): The base URL of the Ollama server.

    Returns:
        tuple: A tuple containing a boolean indicating success and either a list of models or an error message.
    """
    try:
        response = requests.get(f"{base_url}/api/tags")
        response.raise_for_status()
        data = response.json()
        models = [model["name"] for model in data.get("models", [])]
        return True, models
    except requests.exceptions.ConnectionError:
        return False, "Failed to connect to Ollama server."
    except requests.exceptions.RequestException as e:
        return False, f"Error retrieving models: {e}"
    except (KeyError, ValueError) as e:
        return False, f"Error parsing response: {e}"

In [ ]:
connected, message, version = check_ollama_connectivity()
print(version, message)

In [ ]:
success, models = get_available_models()
print(models)

## Basic Generation

In [ ]:
# Create client
client = OllamaClient()

# Generate response
response = client.generate("What is the capital of France?")
print(response)

## Streaming Generation

In [ ]:
# Generate streaming response
for chunk in client.generate("Tell me a short story", stream=True):
    print(chunk, end="", flush=True)
print()

## Parameter Configuration

In [ ]:
# Generate with custom parameters
response = client.generate(
    "Write a creative story",
    temperature=0.9,  # More creative
    top_p=0.95,  # More diverse outputs
)
print(response)

## Error Handling

In [ ]:
# Test with invalid model
response = client.generate("Hello", model="invalid-model")
print(response)